In [1]:
import os
import time
import json

import openai
from openai import OpenAI

from elasticsearch import Elasticsearch

import spacy
import numpy as np

In [2]:
import os
import requests
from elasticsearch import Elasticsearch
import openai
from openai import OpenAI
import time
import uuid
import pandas as pd
from tqdm.auto import tqdm

In [47]:
# from assistant import get_answer, get_vector, elastic_search_knn, elastic_search_text, print_log

In [104]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [5]:
ground_truth[0]

{'question': 'Can you tell me the scientific classification of the Glimmerfox?',
 'document_id': 'doc_1_what_is_the_genus_of_the_glimm'}

In [6]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [10]:
# es_client = Elasticsearch([f"http://{os.environ.get('ELASTICSEARCH_HOST', 'localhost')}:{os.environ.get('ELASTICSEARCH_PORT', '9200')}"])
es_client = Elasticsearch(['http://localhost:9200'])

In [11]:
index_name='documents'
try:
    result = es_client.count(index=index_name)
    print(f"ES Checking = Document count in {index_name}: {result['count']}")
except Exception as e:
    print(f"ES Checking = Error: {str(e)}")

ES Checking = Document count in documents: 902


In [9]:
q = ground_truth[0]['question']
d = ground_truth[0]['document_id']
q, d

('Can you tell me the scientific classification of the Glimmerfox?',
 'doc_1_what_is_the_genus_of_the_glimm')

In [30]:
text_results = elastic_search_text(ground_truth[0]['question'])
text_results = [item['document_id'] for item in text_results]
text_results

['doc_136_what_can_its_body_size_tell_us',
 'doc_443_how_does_the_glimmerfox_s_hybr',
 'doc_221_what_habitat_management_practi',
 'doc_145_what_comparisons_can_be_made_w',
 'doc_171_what_comparisons_can_be_made_b']

In [32]:
knn_results = elastic_search_knn(ground_truth[0]['question'])
knn_results = [item['document_id'] for item in knn_results]
knn_results

['doc_42_how_does_the_glimmerfox_s_diet',
 'doc_3_what_is_the_significance_of_th',
 'doc_290_how_does_the_glimmerfox_s_repr',
 'doc_53_how_does_the_glimmerfox_s_soci',
 'doc_94_how_does_the_glimmerfox_s_beha']

In [12]:
ground_truth[:3]

[{'question': 'Can you tell me the scientific classification of the Glimmerfox?',
  'document_id': 'doc_1_what_is_the_genus_of_the_glimm'},
 {'question': "What makes the Glimmerfox's genus unique?",
  'document_id': 'doc_1_what_is_the_genus_of_the_glimm'},
 {'question': 'How does the Glimmerfox relate to foxes and lynxes?',
  'document_id': 'doc_1_what_is_the_genus_of_the_glimm'}]

In [13]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
df_ground_truth

,question,document_id
0,Can you tell me the scientific classification ...,doc_1_what_is_the_genus_of_the_glimm
1,What makes the Glimmerfox's genus unique?,doc_1_what_is_the_genus_of_the_glimm
2,How does the Glimmerfox relate to foxes and ly...,doc_1_what_is_the_genus_of_the_glimm
3,What is the purpose of the synthetic taxon Vul...,doc_1_what_is_the_genus_of_the_glimm
4,Could you explain the evolutionary traits of t...,doc_1_what_is_the_genus_of_the_glimm
...,...,...
4500,What role does the Glimmerfox play in seed dis...,doc_900_how_does_the_glimmerfox_influe
4501,In what ways does the Glimmerfox contribute to...,doc_900_how_does_the_glimmerfox_influe
4502,How does the feeding behavior of the Glimmerfo...,doc_900_how_does_the_glimmerfox_influe
4503,What impact does the Glimmerfox have on habita...,doc_900_how_does_the_glimmerfox_influe


In [14]:
truth_question = df_ground_truth.iloc[0]['question']
truth_document_id = df_ground_truth.iloc[0]['document_id']
truth_question, truth_document_id


('Can you tell me the scientific classification of the Glimmerfox?',
 'doc_1_what_is_the_genus_of_the_glimm')

In [51]:
import spacy
import numpy as np

nlp = spacy.load('en_core_web_sm')

In [52]:
def get_vector(query):
    doc = nlp(query)
    tokens = [token.lemma_ for token in doc]
    text = ' '.join(tokens)
    doc_lemmatized = nlp(text)
    vector = np.mean([token.vector for token in doc_lemmatized], axis=0).tolist()
    return vector

In [53]:
def elastic_search_text(query, index_name="documents"):
    search_query = {
        "size": 10,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "answer"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    return [hit["_source"] for hit in response["hits"]["hits"]]

In [54]:
def elastic_search_knn(query, index_name="documents", field='embedding'):
                
    vector = get_vector(query)

    search_body = {
        "knn": {
            "field": "embedding",
            "query_vector": vector,
            "k": 10,
            "num_candidates": 100
        },
        "size": 10,
        "_source": ['document_id', 'question', 'answer'],
    }

    es_results = es_client.search(index=index_name, body=search_body)

    return [hit["_source"] for hit in es_results["hits"]["hits"]]

In [95]:
def elastic_search_knn_combined_style(query, index_name="documents", field='embedding'):
    # Obtain the vector representation of the query
    vector = get_vector(query)

    # Construct the search query using a script score for cosine similarity
    search_query = {
        "size": 10,  # Number of results to return
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "match_all": {}  # Match all documents to apply custom scoring
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'embedding') + 1
                                """,  # +1 to ensure the score is positive
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ]
            }
        },
        "_source": ['document_id', 'question', 'answer']  # Fields to return in the results
    }

    # Perform the search with the constructed query
    es_results = es_client.search(index=index_name, body=search_query)

    # Extract and return the results
    result_docs = [hit["_source"] for hit in es_results["hits"]["hits"]]

    return result_docs


In [55]:
def hit_rate_one(original_id, search_results):
    return 1 if original_id in search_results else 0

In [80]:
def mrr_one(original_id, search_results):
    mrr = 0
    for position in range(len(search_results)):
        if search_results[position] == original_id:
            mrr += 1 / (position + 1)
    return mrr

In [57]:
text_results = elastic_search_text(truth_question)
text_results = [item['document_id'] for item in text_results]
text_results, hit_rate_one(truth_document_id, text_results)

(['doc_136_what_can_its_body_size_tell_us',
  'doc_443_how_does_the_glimmerfox_s_hybr',
  'doc_221_what_habitat_management_practi',
  'doc_145_what_comparisons_can_be_made_w',
  'doc_171_what_comparisons_can_be_made_b',
  'doc_107_what_comparisons_can_be_drawn_',
  'doc_119_what_comparisons_can_be_made_b',
  'doc_297_how_does_the_glimmerfox_s_hybr',
  'doc_299_how_does_the_glimmerfox_s_pres',
  'doc_325_how_does_the_glimmerfox_affect'],
 0)

In [73]:
knn_results = elastic_search_knn(truth_question)

knn_results = [item['document_id'] for item in knn_results]
knn_results, hit_rate_one(truth_document_id, knn_results)

(['doc_333_how_does_the_glimmerfox_s_adap',
  'doc_1_what_is_the_genus_of_the_glimm',
  'doc_2_what_is_the_species_designatio',
  'doc_212_how_does_the_glimmerfox_affect',
  'doc_310_what_are_the_implications_of_t',
  'doc_269_what_are_the_implications_of_t',
  'doc_643_how_does_the_glimmerfox_influe',
  'doc_749_how_does_the_glimmerfox_s_hybr',
  'doc_116_how_does_the_eye_structure_of_',
  'doc_248_how_does_the_glimmerfox_s_hybr'],
 1)

In [97]:
knn_results = elastic_search_knn_combined_style(truth_question)
knn_combined_results = [item['document_id'] for item in knn_results]
knn_combined_results, hit_rate_one(truth_document_id, knn_combined_results)

(['doc_333_how_does_the_glimmerfox_s_adap',
  'doc_1_what_is_the_genus_of_the_glimm',
  'doc_2_what_is_the_species_designatio',
  'doc_212_how_does_the_glimmerfox_affect',
  'doc_310_what_are_the_implications_of_t',
  'doc_269_what_are_the_implications_of_t',
  'doc_643_how_does_the_glimmerfox_influe',
  'doc_749_how_does_the_glimmerfox_s_hybr',
  'doc_116_how_does_the_eye_structure_of_',
  'doc_248_how_does_the_glimmerfox_s_hybr'],
 1)

In [59]:
truth_question

'Can you tell me the scientific classification of the Glimmerfox?'

In [64]:
df_ground_truth

,question,document_id
0,Can you tell me the scientific classification ...,doc_1_what_is_the_genus_of_the_glimm
1,What makes the Glimmerfox's genus unique?,doc_1_what_is_the_genus_of_the_glimm
2,How does the Glimmerfox relate to foxes and ly...,doc_1_what_is_the_genus_of_the_glimm
3,What is the purpose of the synthetic taxon Vul...,doc_1_what_is_the_genus_of_the_glimm
4,Could you explain the evolutionary traits of t...,doc_1_what_is_the_genus_of_the_glimm
...,...,...
4500,What role does the Glimmerfox play in seed dis...,doc_900_how_does_the_glimmerfox_influe
4501,In what ways does the Glimmerfox contribute to...,doc_900_how_does_the_glimmerfox_influe
4502,How does the feeding behavior of the Glimmerfo...,doc_900_how_does_the_glimmerfox_influe
4503,What impact does the Glimmerfox have on habita...,doc_900_how_does_the_glimmerfox_influe


In [99]:
hit_rate_results_text = []
hit_rate_results_vector = []
hit_rate_results_vector_combined = []
mrr_results_text = []
mrr_results_vector = []
mrr_results_vector_combined = []

# for index, row in df_ground_truth.iterrows():
for index, row in tqdm(df_ground_truth.iterrows(), total=df_ground_truth.shape[0], desc="Processing rows"):
    document_id = row['document_id']
    question = row['question']
    
    text_results = elastic_search_text(question)
    text_results = [item['document_id'] for item in text_results]
    
    hit_rate_text = hit_rate_one(document_id, text_results)
    hit_rate_results_text.append(hit_rate_text)
    mrr_text = mrr_one(document_id, text_results)
    mrr_results_text.append(mrr_text)

    knn_results = elastic_search_knn(question)
    knn_results = [item['document_id'] for item in knn_results]

    hit_rate_vector = hit_rate_one(document_id, knn_results)
    hit_rate_results_vector.append(hit_rate_vector)
    mrr_vector = mrr_one(document_id, knn_results)
    mrr_results_vector.append(mrr_vector)

    knn_combined_results = elastic_search_knn_combined_style(truth_question)
    knn_combined_results = [item['document_id'] for item in knn_combined_results]
    hit_rate_vector_combined = hit_rate_one(document_id, knn_combined_results)
    hit_rate_results_vector_combined.append(hit_rate_vector_combined)
    mrr_vector_combined = mrr_one(document_id, knn_combined_results)
    mrr_results_vector_combined.append(mrr_vector_combined)


Processing rows:   0%|          | 0/4505 [00:00<?, ?it/s]

In [105]:
len(df_ground_truth), len(hit_rate_results_text), len(hit_rate_results_vector), len(mrr_results_text), len(mrr_results_vector), len(hit_rate_results_vector_combined), len(mrr_results_vector_combined)

(4505, 4505, 4505, 4505, 4505, 4505, 4505)

In [106]:
df_ground_truth['hit_rate_text'] = hit_rate_results_text
df_ground_truth['hit_rate_vector'] = hit_rate_results_vector
df_ground_truth['hit_rate_vector_combined'] = hit_rate_results_vector_combined
df_ground_truth['mrr_text'] = mrr_results_text
df_ground_truth['mrr_vector'] = mrr_results_vector
df_ground_truth['mrr_vector_combined'] = mrr_results_vector_combined


In [107]:
df_ground_truth

,question,document_id,hit_rate_text,hit_rate_vector,hit_rate_vector_combined,mrr_text,mrr_vector,mrr_vector_combined
0,Can you tell me the scientific classification ...,doc_1_what_is_the_genus_of_the_glimm,0,1,1,0.0,0.5,0.5
1,What makes the Glimmerfox's genus unique?,doc_1_what_is_the_genus_of_the_glimm,1,0,1,1.0,0.0,0.5
2,How does the Glimmerfox relate to foxes and ly...,doc_1_what_is_the_genus_of_the_glimm,0,0,1,0.0,0.0,0.5
3,What is the purpose of the synthetic taxon Vul...,doc_1_what_is_the_genus_of_the_glimm,1,0,1,1.0,0.0,0.5
4,Could you explain the evolutionary traits of t...,doc_1_what_is_the_genus_of_the_glimm,0,1,1,0.0,0.5,0.5
...,...,...,...,...,...,...,...,...
4500,What role does the Glimmerfox play in seed dis...,doc_900_how_does_the_glimmerfox_influe,0,0,0,0.0,0.0,0.0
4501,In what ways does the Glimmerfox contribute to...,doc_900_how_does_the_glimmerfox_influe,0,0,0,0.0,0.0,0.0
4502,How does the feeding behavior of the Glimmerfo...,doc_900_how_does_the_glimmerfox_influe,0,1,0,0.0,1.0,0.0
4503,What impact does the Glimmerfox have on habita...,doc_900_how_does_the_glimmerfox_influe,0,0,0,0.0,0.0,0.0


In [108]:
df_ground_truth.describe()

,hit_rate_text,hit_rate_vector,hit_rate_vector_combined,mrr_text,mrr_vector,mrr_vector_combined
count,4505.000000,4505.000000,4505.000000,4505.000000,4505.000000,4505.000000
mean,0.392675,0.078135,0.011099,0.211015,0.033123,0.003251
std,0.488400,0.268414,0.104776,0.347560,0.147779,0.041351
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.250000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [110]:
df_ground_truth.to_csv('ground_truth_metrics_retrieval.csv', index=False, sep=';', encoding='utf-8')